# Service 3.2.2 - Optimization Service

In [34]:
import logging

import pandas as pd
import glob
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from pymoo.problems.functional import FunctionalProblem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
from pymoo.termination.default import DefaultSingleObjectiveTermination


## Variables

In [27]:
LHV = 37411 # kj/m3
LHV_ng = LHV # Used for conversion from m3/hr to 
eta_lim = 1.3 
zeros = 1
random_seed = 1

scaler = StandardScaler()

## Dataframe Creation

In [22]:
# Creazione del dataframe

def create_input(path, save_local_file, **file_format):

    "Creazione DataFrame Completo"

    #Genera una lista fatta da tutti i nomi che rientrano nella richiesta 

    nomifiles=(glob.glob(path))

    df=pd.DataFrame()

    for nomi in nomifiles:
        A0=pd.read_csv(nomi, sep=';', header=None)
        df=pd.concat([df,A0])


    nomi_originali = df.iloc[:,2].unique() #Vediamo quante grandezze vengono studiate

    # elimina le colonne 'a' e 'b' dal dataframe
    df=df.drop(df.columns[0],axis=1)
    df=df.drop(df.columns[0],axis=1)


    df.columns = ['nome', 'orario', 'valore'] #Cambiamo il nome delle colonne

    df['valore']=df['valore'].str.replace(',', '.') #Aggiustiamo i valori del dataframe 
    df['valore']=df['valore'].str.strip() #Serve per togliere tutti gli spazi da quella colonna
    df['valore']=df['valore'].astype(float) #Rendiamo la colonna dei numeri float


    # Crea un nuovo dataframe con gli orari come prima colonna

    df = df.pivot(index='orario', columns='nome', values='valore')

    df.index = pd.to_datetime(df.index)

    # Reimposta l'indice

    df_30 = df.resample('30T').mean()
    # df_30=df.resample('15T').interpolate()


    #Limitati al temo di funzionamento B1-2

    df_30 = df_30.loc['2023-06-07 00:00:00':'2023-10-17 19:00:00']

    dataset = df_30.copy()

    dataset['NG Consumption [kW]'] = dataset['CONSUMO GAS (30 minutos)'].diff()*(LHV/1800)

    # dataset['NG Consumption [kW]'] = dataset['NG Consumption [kW]'].shift(-1)

    dataset['eta'] = dataset['ENERGIA INSTANTANEA (15 minuto)']/(dataset['NG Consumption [kW]']+0.001)
    dataset['Boiler 1 Hours'] = dataset['Horas Funcionamiento Caldera 1 (15 minuto)'].diff()
    dataset['Boiler 2 Hours'] = dataset['Horas Funcionamiento Caldera 2 (15 minuto)'].diff()
    dataset['Boiler 3 Hours'] = dataset['Horas Funcionamiento Caldera 3 (15 minuto)'].diff()
    dataset['Boiler 3 Hours']=dataset['Boiler 3 Hours'].replace(np.nan, 0)

    dataset['BH']=dataset['Boiler 1 Hours']+dataset['Boiler 2 Hours']

    dataset=pd.DataFrame(dataset)

    if zeros==1:
        #Elimino gli zeri da boiler hours
        dataset['filter'] = dataset.apply(lambda row: 0 if row['eta'] < eta_lim and
                                        row['ENERGIA INSTANTANEA (15 minuto)'] > 50
                                        and row['NG Consumption [kW]'] > 50
                                        #and row['BH'] > 0.05
                                        else 1, axis=1) #applica il se
    else:
        #Filtro ma lascio gli zeri
        dataset['filter'] = dataset.apply(lambda row: 0 if row['eta'] < eta_lim else 1, axis=1) #applica il se

    #Elimino i nan

    dataset = dataset.loc[dataset['filter'] != 1]
    dataset=dataset.drop('BH', axis=1)
    dataset=dataset.drop('filter', axis=1)

    dataset.fillna(0, inplace=True)

    if (save_local_file == True and file_format == 'xlsx'):
        dataset.to_excel('TrainingDataset.xlsx')
    elif (save_local_file == True and file_format == 'csv'):
        dataset.to_excel('TraningDataset.csv')
    else:
        pass


    return df_30, dataset

dataset = create_input('resources/RVENA_23*.csv', save_local_file=False)[1]
dataset.head()


nome,CONSUMO GAS (30 minutos),ENERGIA ACUMULADA (30 minutos),ENERGIA INSTANTANEA (15 minuto),Horas Funcionamiento Caldera 1 (15 minuto),Horas Funcionamiento Caldera 2 (15 minuto),Horas Funcionamiento Caldera 3 (15 minuto),TEMP IMP CALDERA 1 (15 minuto),TEMP IMP CALDERA 2 (15 minuto),TEMP IMP CALDERA 3 (15 minuto),TEMP IMP CALDERAS (15 minuto),TEMP RET CALDERAS (15 minuto),TEMPERATURA IMPULSION ANILLO (15 minuto),TEMPERATURA RETORNO ANILLO (15 minuto),VOLUMEN ACUMULADO (15 minuto),VOLUMEN INSTANTANEO (15 minuto),NG Consumption [kW],eta,Boiler 1 Hours,Boiler 2 Hours,Boiler 3 Hours
orario,,,,,,,,,,,,,,,,,,,,
2023-06-07 06:00:00,4518144.0,82838200.0,740.605,23398.18,23341.42,0.0,64.050,57.395,73.145,72.320,71.300,73.815,51.805,8215091.30,28.945,1517.223889,0.488131,0.0,0.0,0.0
2023-06-07 09:00:00,4518192.0,82838896.0,427.275,23398.18,23341.42,0.0,64.440,56.855,70.305,70.405,63.795,64.815,54.905,8215226.17,39.855,789.787778,0.540999,0.0,0.0,0.0
2023-06-07 14:00:00,4518257.0,82839800.0,954.240,23398.18,23341.42,0.0,63.275,55.940,68.245,68.360,60.795,62.665,50.775,8215630.78,56.220,1330.168889,0.717382,0.0,0.0,0.0
2023-06-07 17:30:00,4518322.0,82840704.0,968.480,23398.18,23341.42,0.0,61.850,55.030,68.760,68.635,61.360,63.445,51.500,8215765.65,50.765,1330.168889,0.728087,0.0,0.0,0.0
2023-06-07 20:30:00,4518376.0,82841504.0,1068.180,23398.18,23341.42,0.0,61.850,54.115,75.200,74.095,72.615,74.010,53.360,8215900.52,45.670,1101.546111,0.969709,0.0,0.0,0.0


## ANN Model

In [36]:
def MLModel():

    X = dataset.loc[:,['ENERGIA INSTANTANEA (15 minuto)','TEMP IMP CALDERAS (15 minuto)']]  #Le x e y della mia F
    y = dataset.loc[:,['eta']]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_seed)
    scaler_fitted = scaler.fit(X_train)
    X_train = scaler_fitted.transform(X_train)
    X_test = scaler_fitted.transform(X_test)

    # Definition of the ANN Model

    model = MLPRegressor(hidden_layer_sizes=(20, 100,300,100, 20),
                        max_iter=100000000,
                        verbose=True,
                        solver='adam',
                        learning_rate='adaptive',
                        random_state=random_seed,
                        activation='relu')
    
    model.fit(X_train, y_train)

    # Valutazione delle prestazioni del modello sui dati di test

    score = model.score(X_test, y_test)
    print(f'R^2 score: {score:.2f}')

    # Utilizzo del modello per fare previsioni sui dati di test

    X_pred = scaler.transform(X)    #Utilizzo lo stesso scaler che è stato fittato prima
    y_pred = model.predict(X_pred)

    return model, score

model = MLModel()[0]
print(MLModel()[1])


c:\Users\annatalini\.virtualenvs\TestDevelopementAPI-8CIef_ba\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Iteration 1, loss = 0.02670300
Iteration 2, loss = 0.01454046
Iteration 3, loss = 0.01112334
Iteration 4, loss = 0.01043112
Iteration 5, loss = 0.00986917
Iteration 6, loss = 0.00914802
Iteration 7, loss = 0.00864435
Iteration 8, loss = 0.00838917
Iteration 9, loss = 0.00831613
Iteration 10, loss = 0.00803582
Iteration 11, loss = 0.00789528
Iteration 12, loss = 0.00775230
Iteration 13, loss = 0.00763683
Iteration 14, loss = 0.00751671
Iteration 15, loss = 0.00744461
Iteration 16, loss = 0.00745974
Iteration 17, loss = 0.00734537
Iteration 18, loss = 0.00746654
Iteration 19, loss = 0.00749367
Iteration 20, loss = 0.00722664
Iteration 21, loss = 0.00734083
Iteration 22, loss = 0.00726054
Iteration 23, loss = 0.00705062
Iteration 24, loss = 0.00710073
Iteration 25, loss = 0.00710347
Iteration 26, loss = 0.00700830
Iteration 27, loss = 0.00702521
Iteration 28, loss = 0.00694483
Iteration 29, loss = 0.00690769
Iteration 30, loss = 0.00690857
Iteration 31, loss = 0.00685209
Iteration 32, los

c:\Users\annatalini\.virtualenvs\TestDevelopementAPI-8CIef_ba\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Iteration 2, loss = 0.01454046
Iteration 3, loss = 0.01112334
Iteration 4, loss = 0.01043112
Iteration 5, loss = 0.00986917
Iteration 6, loss = 0.00914802
Iteration 7, loss = 0.00864435
Iteration 8, loss = 0.00838917
Iteration 9, loss = 0.00831613
Iteration 10, loss = 0.00803582
Iteration 11, loss = 0.00789528
Iteration 12, loss = 0.00775230
Iteration 13, loss = 0.00763683
Iteration 14, loss = 0.00751671
Iteration 15, loss = 0.00744461
Iteration 16, loss = 0.00745974
Iteration 17, loss = 0.00734537
Iteration 18, loss = 0.00746654
Iteration 19, loss = 0.00749367
Iteration 20, loss = 0.00722664
Iteration 21, loss = 0.00734083
Iteration 22, loss = 0.00726054
Iteration 23, loss = 0.00705062
Iteration 24, loss = 0.00710073
Iteration 25, loss = 0.00710347
Iteration 26, loss = 0.00700830
Iteration 27, loss = 0.00702521
Iteration 28, loss = 0.00694483
Iteration 29, loss = 0.00690769
Iteration 30, loss = 0.00690857
Iteration 31, loss = 0.00685209
Iteration 32, loss = 0.00684775
Iteration 33, lo

In [33]:
def ann():
    
    x = pd.DataFrame({'ENERGIA INSTANTANEA (15 minuto)': [E], 'TEMP IMP CALDERAS (15 minuto)': [x]})
    x_scaled = scaler.transform(x)
    prediction = MLModel[0].predict(x_scaled)
    return prediction

print(ann())

NameError: name 'E' is not defined

In [35]:
def print_metrics(y_true, y_pred):

    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    return mae, r2

## Optimization

In [51]:
class Optimizer:

    def __init__(self, dataset, model, n_gen, pop_size):
        
        self.optimization_df = dataset.copy()
        self.X = 1
        self.n = len(dataset)
        self.start_o = 0
        self.final_df = self.optimization_df[self.start_o:self.start_o + self.n]
        self.model = model
        self.fixed_value = 0.5
        self.ngen = n_gen
        self.pop_size = pop_size
            
    def f(self, x):

        # Reshape the decision variables into a matrix with n rows and X columns
        x_matrix = x.reshape((self.n, self.X))
        
        x_matrix = np.hstack((self.final_df['ENERGIA INSTANTANEA (15 minuto)'].values.reshape((self.n, 1)), x_matrix))
        x_matrix = pd.DataFrame(x_matrix, columns=['ENERGIA INSTANTANEA (15 minuto)','TEMP IMP CALDERAS (15 minuto)'])

        
        # Apply the scaler transformation to the decision variables matrix
        x_matrix_scaled = scaler.transform(x_matrix)
        
        # Calculate the sum of the model predictions for all timesteps
        eta=self.model.predict(x_matrix_scaled)
        
        
        f=self.final_df['ENERGIA INSTANTANEA (15 minuto)'].values/eta
        
        return np.sum(f)
    

    def f_values(self, x):

        # Reshape the decision variables into a matrix with n rows and X columns
        x_matrix = x.reshape((self.n, self.X))
        
        x_matrix = np.hstack((self.final_df['ENERGIA INSTANTANEA (15 minuto)'].values.reshape((self.n, 1)), x_matrix))
        x_matrix = pd.DataFrame(x_matrix, columns=['ENERGIA INSTANTANEA (15 minuto)','TEMP IMP CALDERAS (15 minuto)'])

        
        # Apply the scaler transformation to the decision variables matrix
        x_matrix_scaled = scaler.transform(x_matrix)
        
        # Calculate the sum of the model predictions for all timesteps
        eta=self.model.predict(x_matrix_scaled)
        
        eta = np.clip(eta, a_min=None, a_max=1.3)

        f=self.final_df['ENERGIA INSTANTANEA (15 minuto)'].values/eta
    
        return f

    def g1(self, x):
        # Reshape the decision variables into a matrix with n rows and X columns
        x_matrix = x.reshape((self.n, self.X))
        
        # Calculate the constraint values for each timestep
        g = x_matrix[:, 2] - x_matrix[:, 1]
        g=np.max(g, axis=0)
        
        return g
    


    def optimize(self):
            
        self.termination = DefaultSingleObjectiveTermination(xtol=1e-800, cvtol=1e-600, ftol=0.05, period=200, n_max_gen=self.ngen, n_max_evals=1000000000)
        algorithm = NSGA2(pop_size=self.pop_size)
        self.best_objective_values = []  

        def callback(algorithm):

            print(f"Generation: {(100*algorithm.n_gen/self.ngen):.2f}%")
            best_objective_value = algorithm.pop.get("F").min()
            self.best_objective_values.append(best_objective_value)

        self.problem = FunctionalProblem(self.X * self.n, self.f, constr_ieq=[], xl=60, xu=90)

        res = minimize(self.problem, algorithm, self.termination, seed=1, callback = callback)

        self.gas_real = self.f(np.array(dataset.loc[:,['TEMP IMP CALDERAS (15 minuto)']]))
        self.optimized_gas = res.F
        self.temperature = res.X.reshape(self.n,self.X)

        df_solutions = self.f_values(res.X)

        solution = {'Strategy':{
            'realGas': self.gas_real,
            'OptimizedGas': self.optimized_gas,
            'Saved Gas': f'{(self.gas_real-self.optimized_gas)/2} kW/h',
            'Saved Cost': f'{100*(1-self.optimized_gas/self.gas_real)} %'

        }}

        return solution, df_solutions

In [53]:
print(Optimizer(dataset, model, 100, 200).optimize()[0])

Generation: 1.00%
Generation: 2.00%
Generation: 3.00%
Generation: 4.00%
Generation: 5.00%
Generation: 6.00%
Generation: 7.00%
Generation: 8.00%
Generation: 9.00%
Generation: 10.00%
Generation: 11.00%
Generation: 12.00%
Generation: 13.00%
Generation: 14.00%
Generation: 15.00%
Generation: 16.00%
Generation: 17.00%
Generation: 18.00%
Generation: 19.00%
Generation: 20.00%
Generation: 21.00%
Generation: 22.00%
Generation: 23.00%
Generation: 24.00%
Generation: 25.00%
Generation: 26.00%
Generation: 27.00%
Generation: 28.00%
Generation: 29.00%
Generation: 30.00%
Generation: 31.00%
Generation: 32.00%
Generation: 33.00%
Generation: 34.00%
Generation: 35.00%
Generation: 36.00%
Generation: 37.00%
Generation: 38.00%
Generation: 39.00%
Generation: 40.00%
Generation: 41.00%
Generation: 42.00%
Generation: 43.00%
Generation: 44.00%
Generation: 45.00%
Generation: 46.00%
Generation: 47.00%
Generation: 48.00%
Generation: 49.00%
Generation: 50.00%
Generation: 51.00%
Generation: 52.00%
Generation: 53.00%
Ge

## Plots